### Voy a intentar descargar el taco, y homologarlo a yolo, porque vienen en coco. Tengo la descarga de github oficial TACO en la ruta

In [21]:
#!pip install tqdm

In [6]:
#Scrip de GPT para descargar las imágenes, y homologarlas a YOLO

import os
import json
import requests
from tqdm import tqdm  # Para mostrar progreso en la descarga



In [22]:
# Rutas
coco_json_path = "/content/TACO-master/data/annotations.json"  # Ruta del archivo COCO
output_dir = "TACO-master/data/images/"  # Carpeta donde se guardarán las imágenes

# Crear carpeta si no existe
os.makedirs(output_dir, exist_ok=True)

# Cargar anotaciones COCO
with open(coco_json_path, "r") as f:
    coco_data = json.load(f)

# Descargar imágenes
for img in tqdm(coco_data["images"], desc="Descargando imágenes"):
    img_url = img["flickr_url"]  # En TACO, las imágenes son URLs
    img_id = str(img["id"])
    #img_filename = os.path.join(output_dir, os.path.basename(img_url))

    # Obtener la extensión del archivo desde la URL
    file_extension = os.path.splitext(img_url)[1]  # Obtiene la extensión (.jpg, .png, etc.)


    img_filename = os.path.join(output_dir, img_id + file_extension)

    # Descargar solo si no existe
    if not os.path.exists(img_filename):
        try:
            response = requests.get(img_url, stream=True)
            if response.status_code == 200:
                with open(img_filename, "wb") as f:
                    for chunk in response.iter_content(1024):
                        f.write(chunk)
        except Exception as e:
            print(f"Error descargando {img_url}: {e}")

print("Descarga completada.")

FileNotFoundError: [Errno 2] No such file or directory: '/content/TACO-master/data/annotations.json'

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Tenemos las imágenes en TACO-master/data/images   Vamos a homologar de COCO a YOLO
#ESTE NOOOOOOO PROXIMA CELDA UNO NUEVO QUE DEJA LOS ARCHIVOS CON EL MISMO NOMBRE
# Rutas

images_dir = "TACO-master/data/images/"  # Carpeta donde están las imágenes
labels_dir = "TACO-master/data/labels/"  # Carpeta donde guardaremos las etiquetas YOLO

# Crear carpeta de etiquetas si no existe
os.makedirs(labels_dir, exist_ok=True)

# Cargar anotaciones COCO
with open(coco_json_path, "r") as f:
    coco_data = json.load(f)

# Mapeo de categorías (COCO usa IDs desde 1, YOLO desde 0)
category_map = {cat["id"]: i for i, cat in enumerate(coco_data["categories"])}

# Procesar cada imagen
for img in coco_data["images"]:
    image_id = img["id"]
    img_width = img["width"]
    img_height = img["height"]
    img_filename = img["file_name"]  # Puede ser útil para verificar coincidencias

    label_file_path = os.path.join(labels_dir, f"{image_id}.txt")

    # Filtrar anotaciones de esta imagen
    annotations = [ann for ann in coco_data["annotations"] if ann["image_id"] == image_id]

    if not annotations:
        continue  # Si la imagen no tiene anotaciones, la ignoramos

    with open(label_file_path, "w") as label_file:
        for ann in annotations:
            category_id = category_map[ann["category_id"]]
            x_min, y_min, width, height = ann["bbox"]

            # Convertir a formato YOLO
            x_center = (x_min + width / 2) / img_width
            y_center = (y_min + height / 2) / img_height
            norm_width = width / img_width
            norm_height = height / img_height

            # Guardar en archivo de texto
            label_file.write(f"{category_id} {x_center:.6f} {y_center:.6f} {norm_width:.6f} {norm_height:.6f}\n")

print("Conversión a formato YOLO completada.")

Conversión a formato YOLO completada.


In [11]:
import os
import json

# Rutas
coco_json_path = "TACO-master/data/annotations.json"  # Ruta del archivo COCO
images_dir = "TACO-master/data/images/"  # Carpeta donde están las imágenes
labels_dir = "TACO-master/data/labels/"  # Carpeta donde guardaremos las etiquetas YOLO

# Crear carpeta de etiquetas si no existe
os.makedirs(labels_dir, exist_ok=True)

# Cargar anotaciones COCO
with open(coco_json_path, "r") as f:
    coco_data = json.load(f)

# Mapeo de categorías (COCO usa IDs desde 1, YOLO desde 0)
category_map = {cat["id"]: i for i, cat in enumerate(coco_data["categories"])}

# Procesar cada imagen
for img in coco_data["images"]:
    image_id = img["id"]
    img_width = img["width"]
    img_height = img["height"]
    img_filename = os.path.splitext(img["file_name"])[0]  # Nombre sin extensión

    label_file_path = os.path.join(labels_dir, f"{img_filename}.txt")

    # Filtrar anotaciones de esta imagen
    annotations = [ann for ann in coco_data["annotations"] if ann["image_id"] == image_id]

    if not annotations:
        continue  # Si la imagen no tiene anotaciones, la ignoramos

    with open(label_file_path, "w") as label_file:
        for ann in annotations:
            category_id = category_map[ann["category_id"]]
            x_min, y_min, width, height = ann["bbox"]

            # Convertir a formato YOLO
            x_center = (x_min + width / 2) / img_width
            y_center = (y_min + height / 2) / img_height
            norm_width = width / img_width
            norm_height = height / img_height

            # Guardar en archivo de texto
            label_file.write(f"{category_id} {x_center:.6f} {y_center:.6f} {norm_width:.6f} {norm_height:.6f}\n")

print("Conversión a formato YOLO completada.")

FileNotFoundError: [Errno 2] No such file or directory: 'TACO-master/data/annotations.json'

In [12]:
# Instalamos ultralytics para el YOLO
!pip install ultralytics


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.5/974.5 kB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 114.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 92.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 92.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstal

In [ ]:
# compurebo su version
import ultralytics
from ultralytics import YOLO

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [13]:
print(ultralytics.__version__)

NameError: name 'ultralytics' is not defined

In [ ]:
# Ruta predeterminada de Ultralytics
model_path = os.path.expanduser('~/.cache/ultralytics/')

# Verificar si el archivo del modelo existe en esa ruta
if os.path.exists(model_path):
    print(f"Los archivos están en: {model_path}")
else:
    print("No se encuentra la carpeta de ultralytics.")

No se encuentra la carpeta de ultralytics.


In [ ]:
print(ultralytics.__path__)

['D:\\Users\\elmon\\anaconda3\\envs\\gpuvirtual2\\lib\\site-packages\\ultralytics']


In [ ]:
cache_path = os.path.expanduser("~/.cache/ultralytics/")

# Verificar si la carpeta existe
if os.path.exists(cache_path):
    print(f"Los modelos se guardan en: {cache_path}")
else:
    print("No se encontró la carpeta de cache en la ubicación esperada.")

No se encontró la carpeta de cache en la ubicación esperada.


In [ ]:
model = YOLO('models/yolo11l.pt') #He tenido problemas para encontrar los archivos

100%|██████████| 49.0M/49.0M [00:01<00:00, 46.3MB/s]


In [ ]:
# prompt: importame para montar mi drive

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
#ESTE NOOOOO
#Vamos a organizar las imagenes para train, val y test

import os
import shutil
import random

# Rutas de las imágenes y etiquetas
images_dir = "TACO-master/data/images/"
labels_dir = "TACO-master/data/labels/"

# Rutas de salida
output_dir = "datasets/TACO-YOLO/"
train_img_dir = os.path.join(output_dir, "images/train/")
val_img_dir = os.path.join(output_dir, "images/val/")
test_img_dir = os.path.join(output_dir, "images/test/")
train_lbl_dir = os.path.join(output_dir, "labels/train/")
val_lbl_dir = os.path.join(output_dir, "labels/val/")
test_lbl_dir = os.path.join(output_dir, "labels/test/")

# Crear carpetas si no existen
for d in [train_img_dir, val_img_dir, test_img_dir, train_lbl_dir, val_lbl_dir, test_lbl_dir]:
    os.makedirs(d, exist_ok=True)

# Obtener lista de imágenes
image_files = [f for f in os.listdir(images_dir) if f.endswith((".jpg", ".png"))]

# Mezclar aleatoriamente
random.shuffle(image_files)

# Dividir en 80% train, 10% val, 10% test
train_split = int(0.8 * len(image_files))
val_split = int(0.9 * len(image_files))

train_files = image_files[:train_split]
val_files = image_files[train_split:val_split]
test_files = image_files[val_split:]

# Función para mover archivos
def move_files(file_list, img_dest, lbl_dest):
    for file in file_list:
        img_path = os.path.join(images_dir, file)
        lbl_path = os.path.join(labels_dir, file.replace(".jpg", ".txt").replace(".png", ".txt"))

        if os.path.exists(img_path):
            shutil.copy(img_path, img_dest)  # Copiar en lugar de mover
            print(f"Imagen copiada: {img_path} -> {img_dest}")
        else:
            print(f"Imagen no encontrada: {img_path}")

        if os.path.exists(lbl_path):
            shutil.copy(lbl_path, lbl_dest)
            print(f"Etiqueta copiada: {lbl_path} -> {lbl_dest}")
        else:
            print(f"Etiqueta no encontrada: {lbl_path}")

# Mover archivos
move_files(train_files, train_img_dir, train_lbl_dir)
move_files(val_files, val_img_dir, val_lbl_dir)
move_files(test_files, test_img_dir, test_lbl_dir)

print("División de dataset completada.")

Imagen copiada: TACO-master/data/images/170.png -> datasets/TACO-YOLO/images/train/
Etiqueta copiada: TACO-master/data/labels/170.txt -> datasets/TACO-YOLO/labels/train/
Imagen copiada: TACO-master/data/images/1134.png -> datasets/TACO-YOLO/images/train/
Etiqueta copiada: TACO-master/data/labels/1134.txt -> datasets/TACO-YOLO/labels/train/
Imagen copiada: TACO-master/data/images/1066.png -> datasets/TACO-YOLO/images/train/
Etiqueta copiada: TACO-master/data/labels/1066.txt -> datasets/TACO-YOLO/labels/train/
Imagen copiada: TACO-master/data/images/8.png -> datasets/TACO-YOLO/images/train/
Etiqueta copiada: TACO-master/data/labels/8.txt -> datasets/TACO-YOLO/labels/train/
Imagen copiada: TACO-master/data/images/174.png -> datasets/TACO-YOLO/images/train/
Etiqueta copiada: TACO-master/data/labels/174.txt -> datasets/TACO-YOLO/labels/train/
Imagen copiada: TACO-master/data/images/1165.png -> datasets/TACO-YOLO/images/train/
Etiqueta copiada: TACO-master/data/labels/1165.txt -> datasets/TA

In [ ]:
import os
import shutil
import random

# Rutas de las imágenes y etiquetas
images_dir = "TACO-master/data/images/"
labels_dir = "TACO-master/data/labels/"

# Rutas de salida
output_dir = "TACO-YOLO/"
train_img_dir = os.path.join(output_dir, "images/train/")
val_img_dir = os.path.join(output_dir, "images/val/")
test_img_dir = os.path.join(output_dir, "images/test/")
train_lbl_dir = os.path.join(output_dir, "labels/train/")
val_lbl_dir = os.path.join(output_dir, "labels/val/")
test_lbl_dir = os.path.join(output_dir, "labels/test/")

# Crear carpetas si no existen
for d in [train_img_dir, val_img_dir, test_img_dir, train_lbl_dir, val_lbl_dir, test_lbl_dir]:
    os.makedirs(d, exist_ok=True)

# Obtener lista de imágenes
image_files = [f for f in os.listdir(images_dir) if f.endswith((".jpg", ".png"))]

# Mezclar aleatoriamente
random.shuffle(image_files)

# Dividir en 80% train, 10% val, 10% test
train_split = int(0.8 * len(image_files))
val_split = int(0.9 * len(image_files))

train_files = image_files[:train_split]
val_files = image_files[train_split:val_split]
test_files = image_files[val_split:]

# Función para mover archivos

def move_files(file_list, img_dest, lbl_dest, label_prefix):
    for idx, file in enumerate(file_list):
        img_path = os.path.join(images_dir, file)
        lbl_path = os.path.join(labels_dir, f"{idx}.txt")  # Usa los índices como nombres de etiqueta

        if os.path.exists(img_path):
            shutil.copy(img_path, img_dest)  # Copiar en lugar de mover
            print(f"Imagen copiada: {img_path} -> {img_dest}")
        else:
            print(f"Imagen no encontrada: {img_path}")

        if os.path.exists(lbl_path):
            shutil.copy(lbl_path, lbl_dest)
            print(f"Etiqueta copiada: {lbl_path} -> {lbl_dest}")
        else:
            print(f"Etiqueta no encontrada: {lbl_path}")

# Mover archivos
move_files(train_files, train_img_dir, train_lbl_dir, "train")
move_files(val_files, val_img_dir, val_lbl_dir, "val")
move_files(test_files, test_img_dir, test_lbl_dir, "test")

print("División de dataset completada.")

Imagen copiada: TACO-master/data/images/48661109043_40197b57bd_o.png -> TACO-YOLO/images/train/
Etiqueta copiada: TACO-master/data/labels/0.txt -> TACO-YOLO/labels/train/
Imagen copiada: TACO-master/data/images/47856290731_9bcd8c16e9_o.png -> TACO-YOLO/images/train/
Etiqueta copiada: TACO-master/data/labels/1.txt -> TACO-YOLO/labels/train/
Imagen copiada: TACO-master/data/images/33978469458_a0dee6d229_o.png -> TACO-YOLO/images/train/
Etiqueta copiada: TACO-master/data/labels/2.txt -> TACO-YOLO/labels/train/
Imagen copiada: TACO-master/data/images/40888909583_9f9cc88433_o.png -> TACO-YOLO/images/train/
Etiqueta copiada: TACO-master/data/labels/3.txt -> TACO-YOLO/labels/train/
Imagen copiada: TACO-master/data/images/33979041238_6ae1947d59_o.png -> TACO-YOLO/images/train/
Etiqueta copiada: TACO-master/data/labels/4.txt -> TACO-YOLO/labels/train/
Imagen copiada: TACO-master/data/images/47803638652_9cdd3b0f82_o.png -> TACO-YOLO/images/train/
Etiqueta copiada: TACO-master/data/labels/5.txt -

In [ ]:
import torch
print(torch.cuda.is_available())

True


In [ ]:
!nvidia-smi

Wed Apr  2 19:53:44 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 572.83                 Driver Version: 572.83         CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3080 Ti   WDDM  |   00000000:01:00.0 Off |                  N/A |
|  0%   34C    P8             15W /  400W |    1611MiB /  12288MiB |      3%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118
   ---------------------------------------- 0.0/4.0 MB ? eta -:--:--
   -------------------------------------- - 3.9/4.0 MB 23.4 MB/s eta 0:00:01
   ---------------------------------------- 4.0/4.0 MB 20.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.7 GB ? eta -:--:--
   ---------------------------------------- 0.0/2.7 GB ? eta -:--:--
   ---------------------------------------- 0.0/2.7 GB 6.1 MB/s eta 0:07:25
   ---------------------------------------- 0.0/2.7 GB 14.2 MB/s eta 0:03:12
   ---------------------------------------- 0.0/2.7 GB 17.2 MB/s eta 0:02:39
   ---------------------------------------- 0.0/2.7 GB 17.7 MB/s eta 0:02:34
   ---------------------------------------- 0.0/2.7 GB 18.3 MB/s eta 0:02:29
   ---------------------------------------- 0.0/2.7 GB 19.7 MB/s eta 0:02:18
   ---------------------------------------- 0.0/2.7 GB 20.3 MB/s eta 0:02:13
    --------------------------------------

  You can safely remove it manually.


In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"  # Esto desactiva la GPU para TensorFlow

In [ ]:
!pip install numpy<2

El sistema no puede encontrar el archivo especificado.


In [ ]:
import tensorflow as tf
print("Is GPU available for TensorFlow?: ", tf.config.list_physical_devices('GPU'))


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "D:\Users\elmon\anaconda3\envs\gpuvirtual2\lib\runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "D:\Users\elmon\anaconda3\envs\gpuvirtual2\lib\runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "D:\Users\elmon\anaconda3\envs\gpuvirtual2\lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "D:\Users\elmon\anaconda3\envs\gpuvirtual2\lib\site-packages\traitlets\config\application.py", line 1075, in launch_in

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "D:\Users\elmon\anaconda3\envs\gpuvirtual2\lib\runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "D:\Users\elmon\anaconda3\envs\gpuvirtual2\lib\runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "D:\Users\elmon\anaconda3\envs\gpuvirtual2\lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "D:\Users\elmon\anaconda3\envs\gpuvirtual2\lib\site-packages\traitlets\config\application.py", line 1075, in launch_in

AttributeError: _ARRAY_API not found

ImportError: numpy.core._multiarray_umath failed to import

ImportError: numpy.core.umath failed to import

TypeError: Unable to convert function return value to a Python type! The signature was
	() -> handle

In [18]:
import torch
print(torch.cuda.is_available())

True


In [17]:
import os

yaml_path = '/content/drive/MyDrive/pruebaTACO/datasets/TACO-YOLO/taco.yaml'

# Verifica si el archivo existe
if os.path.exists(yaml_path):
    print("El archivo YAML se encuentra en la ruta especificada.")
else:
    print("No se encontró el archivo YAML en la ruta especificada.")

No se encontró el archivo YAML en la ruta especificada.


In [20]:
import yaml

# Lee el archivo YAML
with open(yaml_path, 'r') as file:
    data = yaml.safe_load(file)

# Imprime las rutas de las imágenes
print("Ruta de entrenamiento:", data['train'])
print("Ruta de validación:", data['val'])
print("Ruta de prueba:", data['test'])

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/pruebaTACO/datasets/TACO-YOLO/taco.yaml'

In [ ]:
import json

settings_path = '/root/.config/Ultralytics/settings.json'

# Cargar el archivo settings.json
with open(settings_path, 'r') as f:
    settings = json.load(f)

# Actualizar el campo 'dataset_download_directory' con la ruta a tu Google Drive
settings['dataset_download_directory'] = '/content/drive/MyDrive/pruebaTACO/datasets/'

# Guardar los cambios de vuelta en el archivo
with open(settings_path, 'w') as f:
    json.dump(settings, f, indent=4)

print("Ruta de descarga actualizada correctamente.")

Ruta de descarga actualizada correctamente.


In [16]:
#OJO, necesito pytorch, tensor flow is dead
train_results = model.train(
    data="/content/drive/MyDrive/pruebaTACO/datasets/TACO-YOLO/taco.yaml",  # Path to dataset configuration file
    epochs=100,
    imgsz=640,
    batch=16,
    device=0
)

NameError: name 'model' is not defined

In [ ]:
YOLO detect train data=taco.yaml model=model epochs=50 imgsz=640 batch=16 device=0

SyntaxError: invalid syntax (401659116.py, line 1)

In [ ]:
import os

train_labels = os.listdir("datasets/TACO-YOLO/labels/train/")
val_labels = os.listdir("datasets/TACO-YOLO/labels/val/")

print(f"Etiquetas en train: {len(train_labels)}")
print(f"Etiquetas en val: {len(val_labels)}")

# Muestra algunas etiquetas
print("Ejemplo de archivos en train:", train_labels[:5])
print("Ejemplo de archivos en val:", val_labels[:5])

Etiquetas en train: 827
Etiquetas en val: 103
Ejemplo de archivos en train: ['0.txt', '1.txt', '10.txt', '100.txt', '101.txt']
Ejemplo de archivos en val: ['0.txt', '1.txt', '10.txt', '100.txt', '101.txt']


In [ ]:
import os

# Directorios
train_images_path = "datasets/TACO-YOLO/images/train/"
train_labels_path = "datasets/TACO-YOLO/labels/train/"

# Listar nombres de imágenes y etiquetas (sin extensiones)
train_images = {f.split('.')[0] for f in os.listdir(train_images_path)}
train_labels = {f.split('.')[0] for f in os.listdir(train_labels_path)}

# Comparar nombres
missing_labels = train_images - train_labels
missing_images = train_labels - train_images

print(f"Imágenes sin etiquetas: {len(missing_labels)}")
print(f"Etiquetas sin imágenes: {len(missing_images)}")

# Mostrar algunos casos
print("Ejemplo de imágenes sin etiquetas:", list(missing_labels)[:5])
print("Ejemplo de etiquetas sin imágenes:", list(missing_images)[:5])

Imágenes sin etiquetas: 994
Etiquetas sin imágenes: 827
Ejemplo de imágenes sin etiquetas: ['48580509912_da2cd98a57_o', '48580369326_202d20a772_o', '48630584588_d3d3069e96_o', '48631087367_49ef8daac1_o', '32931539467_325b683a77_o']
Ejemplo de etiquetas sin imágenes: ['475', '297', '497', '379', '479']


In [ ]:
import os

# Directorios
train_images_path = "datasets/TACO-YOLO/images/train/"
train_labels_path = "datasets/TACO-YOLO/labels/train/"

# Obtener nombres de imágenes y etiquetas
train_images = sorted([f for f in os.listdir(train_images_path) if f.endswith(('.jpg', '.png'))])
train_labels = sorted([f for f in os.listdir(train_labels_path) if f.endswith('.txt')])

# Verificar si hay igual cantidad de imágenes y etiquetas
if len(train_images) != len(train_labels):
    print("⚠️ Advertencia: El número de imágenes y etiquetas no coincide.")

# Renombrar etiquetas para que coincidan con las imágenes
for img_name, lbl_name in zip(train_images, train_labels):
    img_base = os.path.splitext(img_name)[0]  # Nombre sin extensión
    lbl_old_path = os.path.join(train_labels_path, lbl_name)
    lbl_new_path = os.path.join(train_labels_path, f"{img_base}.txt")

    os.rename(lbl_old_path, lbl_new_path)
    print(f"Renombrado: {lbl_old_path} -> {lbl_new_path}")

print("✅ Renombrado de etiquetas completado.")

⚠️ Advertencia: El número de imágenes y etiquetas no coincide.
Renombrado: datasets/TACO-YOLO/labels/train/0.txt -> datasets/TACO-YOLO/labels/train/31761220157_927fea50b6_o.txt
Renombrado: datasets/TACO-YOLO/labels/train/1.txt -> datasets/TACO-YOLO/labels/train/32911398817_e6ef283545_o.txt
Renombrado: datasets/TACO-YOLO/labels/train/10.txt -> datasets/TACO-YOLO/labels/train/32911402747_496ac0b26e_o.txt
Renombrado: datasets/TACO-YOLO/labels/train/100.txt -> datasets/TACO-YOLO/labels/train/32911404057_b139c11643_o.txt
Renombrado: datasets/TACO-YOLO/labels/train/101.txt -> datasets/TACO-YOLO/labels/train/32911404467_75881a027e_o.txt
Renombrado: datasets/TACO-YOLO/labels/train/102.txt -> datasets/TACO-YOLO/labels/train/32911404617_b7c9ea03d6_o.txt
Renombrado: datasets/TACO-YOLO/labels/train/103.txt -> datasets/TACO-YOLO/labels/train/32911405727_69e1c9873d_o.txt
Renombrado: datasets/TACO-YOLO/labels/train/104.txt -> datasets/TACO-YOLO/labels/train/32911411497_62842b1d69_o.txt
Renombrado: da

In [ ]:
# PARA VAL

import os

images_val = os.listdir("datasets/TACO-YOLO/images/val")
labels_val = os.listdir("datasets/TACO-YOLO/labels/val")

# Quitar extensiones de imágenes para comparación
image_names = {os.path.splitext(img)[0] for img in images_val}
label_names = {os.path.splitext(lbl)[0] for lbl in labels_val}

# Ver imágenes sin etiquetas
missing_labels = image_names - label_names

if missing_labels:
    print(f"⚠️ {len(missing_labels)} imágenes en val no tienen etiqueta:")
    print(missing_labels)
else:
    print("✅ Todas las imágenes en val tienen su etiqueta correspondiente.")

⚠️ 197 imágenes en val no tienen etiqueta:
{'47803604122_8b9b12d84a_o', '48693735823_46e44e0483_o', '40889612323_86bf8fb981_o', '48631086512_a50cb346ba_o', '47066647264_c1561948ae_o', '46939935055_2b5807711d_o', '47803342522_5c60c6c4e0_o', '48694074286_06d624fb18_o', '47855710851_fb7d20bc58_o', '32912098647_4e3b57777f_o', '48580516777_01a7db04f0_o', '48661106913_d8b8d2fb33_o', '32912020677_04307671cd_o', '40889652193_3f37fd3281_o', '48630584228_82e5d5cd4c_o', '47856358721_83317cd3e7_o', '48700815618_fa6abfa23f_o', '47066085544_a1d7a34a92_o', '40889561193_18325208b2_o', '48421580257_5a6487fc09_o', '47803335992_9c58683430_o', '47856201821_e0c10c6b9d_o', '48693698123_73a7339b12_o', '48694107166_73969cc1f2_o', '48694110611_ebca02ff39_o', '47804039552_c3695ae1fa_o', '32912149767_1a29ed7168_o', '32911605027_cb8da2ed42_o', '48694035346_6a5f7ae33a_o', '40889079393_4dc166cf15_o', '47066742954_899a3cae6b_o', '47855679411_7278f2ba45_o', '33978879488_0055e88548_o', '46939455235_a08e03df35_o', '486

In [ ]:
# prompt: importame para montar mi drive

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 977.3/977.3 kB 37.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 97.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 78.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 54.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 25.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 85.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

In [ ]:
from ultralytics import YOLO
#aml_path = '/content/drive/MyDrive/ColabpruebaTACO/datasets/TACO-YOLO/taco.yaml'

# Load a pretrained YOLO11n model
model = YOLO("/content/drive/MyDrive/last.pt")

# Define path to the image file
source = "/content/drive/MyDrive/IMG_2747.jpeg"

# Run inference on the source
results = model(source)  # list of Results objects


image 1/1 /content/drive/MyDrive/IMG_2747.jpeg: 640x480 1 Glass, 53.7ms
Speed: 3.3ms preprocess, 53.7ms inference, 186.8ms postprocess per image at shape (1, 3, 640, 480)
